#
Simple Training with the 🤗 Transformers Trainer

In [1]:
from datasets import load_dataset
emotion_dataset=load_dataset("emotion")
emotion_dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
emotion_dataset["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [ ]:
emotion_df=emotion_dataset["train"].to_pandas()
emotion_df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [ ]:
features=emotion_dataset["train"].features
features

{'text': Value('string'),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'])}

In [ ]:
id2label={idx:features["label"].int2str(idx) for idx in range(6)}
id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [ ]:
label2id={v:k for k,v in id2label.items()}
label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [ ]:
emotion_df["label"].value_counts(normalize=True).sort_index()

,proportion
label,
0,0.291625
1,0.335125
2,0.081500
3,0.134937
4,0.121063
5,0.035750


# Tokanize all the things

In [ ]:
from transformers import AutoTokenizer
model_ckpt="microsoft/MiniLM-L12-H384-uncased"
tokenizer=AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
tokenizer(emotion_dataset["train"]["text"][:1])

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
def tokenize_text(example):
  return tokenizer(example["text"],truncation=True,max_length=512)

In [ ]:
emotion_dataset=emotion_dataset.map(tokenize_text,batched=True)
emotion_dataset

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

#dealing with class imbalance

In [ ]:
class_weights=(1-(emotion_df["label"].value_counts().sort_index()/len(emotion_df))).values
class_weights

array([0.708375 , 0.664875 , 0.9185   , 0.8650625, 0.8789375, 0.96425  ])

In [ ]:
import torch
class_weights=torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643], device='cuda:0')

In [ ]:
emotion_dataset=emotion_dataset.rename_columns({"label":"labels"})

In [ ]:
from torch import nn
import torch
from transformers import Trainer

class weightsLossTrainer(Trainer):
  def compute_loss(self,model,inputs,return_outputs=False,**kwargs):
    outputs=model(**inputs)
    logits=outputs.get("logits")
    labels=inputs.get("labels")
    loss_fn=nn.CrossEntropyLoss(weight=class_weights)
    loss=loss_fn(logits,labels)
    return (loss,outputs) if return_outputs else loss


In [ ]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                         num_labels=6,
                                                         id2label=id2label,
                                                         label2id=label2id)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: microsoft/MiniLM-L12-H384-uncased
Key               | Status  | 
------------------+---------+-
classifier.weight | MISSING | 
classifier.bias   | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
  labels=pred.label_ids
  preds=pred.predictions.argmax(-1)
  f1=f1_score(labels,preds,average="weighted")
  return {"f1":f1}

In [ ]:
from transformers import TrainingArguments
batch_size=64
logging_steps=len(emotion_dataset["train"])
output_dir="minilm-finetuned-emotion"
training_args=TrainingArguments(output_dir=output_dir,
                                num_train_epochs=5,
                                learning_rate=2e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                eval_strategy="epoch",
                                logging_steps=logging_steps,
                                fp16=True,
                                push_to_hub=True)

In [ ]:
trainer=weightsLossTrainer(model=model,
                            args=training_args,
                            compute_metrics=compute_metrics,
                            train_dataset=emotion_dataset["train"],
                            eval_dataset=emotion_dataset["validation"],
                            processing_class=tokenizer
                            )

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.587962,0.857113
2,No log,0.387824,0.904648
3,No log,0.332334,0.911002
4,No log,0.286461,0.916760
5,No log,0.272274,0.917947


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=1250, training_loss=0.43597646484375, metrics={'train_runtime': 120.7688, 'train_samples_per_second': 662.423, 'train_steps_per_second': 10.35, 'total_flos': 581870908894464.0, 'train_loss': 0.43597646484375, 'epoch': 5.0})

In [ ]:
from transformers import pipeline
model_ckpt="nikk118/minilm-finetuned-emotion"
pipe=pipeline("text-classification",model=model_ckpt)

config.json:   0%|          | 0.00/985 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
pipe("i am really excited about gen AI")

[{'label': 'joy', 'score': 0.9670530557632446}]